In [1]:
import pandas as pd
import re

In [3]:
# Чтение таблицы EA tool

table = pd.read_excel('Копия tech-flflmnt.xlsx', sheet_name='Technologies')
table

,Software Technology,Short Name,Vendor,Technology Code,Company Standard,Категория,Ответственный,номер,технология \n(из опроса),вендор \n(из опроса),архитектор \n(из опроса)
0,((L4bDX5djiZ,sPlan,Abacom,PO000579,Accepted,7.6. Средства автоматизированного проектирован...,NaN,NaN,NaN,NaN,NaN
1,()uk8SSbZPxD,Becker,Becker Mining Systems AG,PO002116,Accepted,7.10.6. Программируемый логический контроллер ...,Галеев Олег Борисович,NaN,NaN,NaN,NaN
2,(0M4JEX5dPA1,FastReport Viewer,Fast Reports,PO000235,Accepted,5.7.1. Средства просмотра электронных документов,Канаков Денис Станиславович,NaN,NaN,NaN,NaN
3,(0Quoxlee5dK,WinBres,Бреслер,PO002749,Approved,7.5.1. Комплексы для мониторинга технического ...,Аритменов Павел Юрьевич,NaN,NaN,NaN,NaN
4,(1M4uEX5djdI,maxView Storage Manager,Microsemi Corporation.,PO000386,Accepted,3.5.10. Средства доступа к общему хранилищу да...,Чадаев Александр Валерьевич,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3012,ZyL4qDX5dLXc,Blackmagic ATEM Switchers,Blackmagic Design,PO000102,Accepted,"5.8.1 Редакторы видео, мультимедиа и анимаций",Канаков Денис Станиславович,NaN,NaN,NaN,NaN
3013,ZyL4vDX5dbGe,CARIS Easy View,CARIS,PO000119,Accepted,7.20.3. Прочее ПО в области строительства здан...,Дилленский Сергей Юрьевич,NaN,NaN,NaN,NaN
3014,ZyukB5UbZzcH,Weintek,Weintek,PO002125,Accepted,7.10.7. Человеко-машинный интерфейс (HMI),Галеев Олег Борисович,NaN,NaN,NaN,NaN
3015,Zzukm0VbZTgM,PACSystems RX3i,GE Digital,PO002123,Accepted,7.10.6. Программируемый логический контроллер ...,Галеев Олег Борисович,NaN,NaN,NaN,NaN


In [4]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [5]:
# Модель эмбеддингов

from FlagEmbedding import BGEM3FlagModel
emb_model = BGEM3FlagModel('/home/victor/gpt1/milvus/embedding/bge-m3', device='cuda', use_fp16=False, normalize_embeddings=True)

/home/victor/.virtualenvs/cvatfo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Нормализация имен вендора и продукта

In [6]:
import re
from transliterate import translit
from copy import deepcopy

# Стоп-слова
RU_STOP = {"и", "в", "во", "не", "на", "но", "при", "для", "к", "из", "от", "с", "со", "о", "а", "у", "по", "над", "под", "до", "без", "или"}
EN_STOP = {"the", "a", "an", "and", "or", "of", "for", "in", "on", "at", "to", "from", "with", "by", "without", "into", "out", "over", "under", "above", "below"}

LEGAL_PREFIXES = [
    r'\bооо\b', r'\bао\b', r'\bзао\b', r'\bип\b', r'\bпао\b', r'\bгк\b',
    r'\bнко\b', r'\bнпо\b', r'\bнпп\b', r'\bнпф\b', r'\bоао\b',
    r'\bltd\b', r'\blimited\b', r'\binc\.?\b', r'\bllc\b', r'\bgmbh\b',
    r'\bcorp\.?\b', r'\bco\.?\b', r'\bsarl\b', r'\bs\.?a\.?\b',
    r'\bplc\b', r'\bgroup\b', r'\bcompany\b', r'\bкомпания\b',
    r'\bиндивидуальный предприниматель\b',
    r'\bобщество с ограниченной ответственностью\b'
]

import re
from transliterate import translit

def is_latin_and_digits_only(s):
    return s.isalnum() and s.isascii()

# Объединённые стоп-слова (в оригинальном написании!)
STOP_WORDS = RU_STOP | EN_STOP

def normalize_vendor_name(name: str) -> str:
    if not isinstance(name, str) or not name.strip():
        return ""
    
    name = name.lower().strip()
    
    # 1. Удалить юридические префиксы
    for pattern in LEGAL_PREFIXES:
        name = re.sub(pattern, ' ', name)
    
    # 2. Удалить всё кроме букв, цифр, пробелов
    name = re.sub(r'[^a-zа-яё0-9\s]', ' ', name)
    
    # 3. Удалить стоп-слова
    words = name.split()
    words = [w for w in words if w and w not in STOP_WORDS]
    name = ' '.join(words)
    
    # 4. Транслитерация (если есть кириллица)
    if re.search(r'[а-яё]', name):
        try:
            name = translit(name, 'ru', reversed=True)
        except Exception:
            pass
        
    name = re.sub(r'\s+', ' ', name).strip()
    
    return name

def normalize_product_name(product: str, vendor_norm: str = "") -> str:
    if not isinstance(product, str) or not product.strip():
        return ""
    
    product = product.strip()
    
    # 1. Удалить версионные паттерны
    version_patterns = [
        r'\b(19|20)\d{2}\b',                    # годы
        r'\b[vV]\.?\d+\.[xX]\b',                # v.4.x, v4.x, V.7.X
        r'\b\d+\.[xX]\b',                       # 8.x
        r'\b[vV]\.?\d+(\.\d+)*[a-z]*\b',        # v.4, v4, v.1.2, v1.2.3a
        r'\b\d+\.\d+(\.\d+)*[a-z]*\b',          # 8.1, 2021.1a
        r'\bR\d+\b',                            # R2
        r'\bSP\d+\b',                           # SP1
        r'\b(x64|x86|64[-\s]?bit|32[-\s]?bit)\b',
    ]
    cleaned = product
    for pat in version_patterns:
        cleaned = re.sub(pat, ' ', cleaned, flags=re.IGNORECASE)
    
    # 2. Удалить пунктуацию
    cleaned = re.sub(r'[^a-zа-яё0-9\s]', ' ', cleaned, flags=re.IGNORECASE)
    cleaned = cleaned.lower()
    
    # 3. Удалить стоп-слова
    words = cleaned.split()
    words = [w for w in words if w and w not in STOP_WORDS]
    cleaned = ' '.join(words)
    
    # 4. Транслитерация
    if re.search(r'[а-яё]', cleaned):
        try:
            cleaned = translit(cleaned, 'ru', reversed=True)
        except Exception:
            pass
    cleaned_copy = deepcopy(cleaned)
    
    # 5. Вычитание vendor_norm
    if vendor_norm.strip():
        vendor_words = set(vendor_norm.lower().split())
        words = [w for w in cleaned.split() if w and w not in vendor_words]
        cleaned = ' '.join(words)
    else:
        cleaned = ' '.join([w for w in cleaned.split() if w])
    
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    # 6. Оставить оригинальное название, если продукт стал пустой
    return cleaned if cleaned else cleaned_copy

# Проверка
vendors = [
    'ООО «1С»',
    'CONUNDRUM INDUSTRIAL LIMITED',
    'ИП Баринова Юлия Дмитриевна',
    'АО «СО ЕЭС»',
    'GE Bently Nevada',
    'ООО ''Контент ИИ''',
    'Custom',
    'Open Source',
    'АО ''НПО ''ЭШЕЛОН''',
]

normalized = [normalize_vendor_name(v) for v in vendors]
print(normalized)

['1s', 'conundrum industrial', 'barinova julija dmitrievna', 'ees', 'ge bently nevada', 'kontent ii', 'custom', 'open source', 'eshelon', '']


# Работа с векторной БД

In [9]:
from pydantic_settings import BaseSettings

class NsiWebAppSettings(BaseSettings):
    SSL_ENABLE: bool=False
    HOST: str = "0.0.0.0"
    PORT: int = 5004
    XINFER_SRV: str = "" #"172.16.33.12:9997"
    MODELS_PATH: str = "models"
    EMBEDDING_MODEL_PATH: str = '/home/victor/gpt1/milvus/embedding' #'/home/victor/chatgpt/embedding'
    SELECT_CUDA_FOR_EMBED_MODEL: bool = True
    SELECT_CUDA_FOR_RERANK_MODEL: bool = True
    LOCAL_CUDA_NUM: int = 1
    LOG_LEVEL: str = 'info'
    LOGDIR: str = "./logs_api"
    LOGGER_NAME: str = "nsi_webapp_logger"
    LOGGER_FILE_NAME: str = "nsi_webapp_logger.log"
    MILVUS_HOST: str = "127.0.0.1" #"172.16.33.32" #"127.0.0.1"
    MILVUS_PORT: str = '20530'
    MILVUS_EMBED_DIM: int = 1024
    MILVUS_ALIAS: str = 'erudit'
    MILVUS_DB: str = 'default'
    WEB_HOST: str = "172.16.33.32"
    WEB_PORT: int = 5003
    WEB_SHARE: bool = False
        
app_settings = NsiWebAppSettings()

In [10]:
from pymilvus import connections, Connections
connections = Connections()

try:
    connections.connect(alias=app_settings.MILVUS_ALIAS, uri=f"https://{app_settings.MILVUS_HOST}:{str(app_settings.MILVUS_PORT)}", 
                        db_name=app_settings.MILVUS_DB, 
                        secure=True,
                        server_pem_path="/home/victor/gpt1/milvus_25/tls/server.pem",
                        server_name="localhost"
                       )
except Exception as e:
    print("Exeption:", str(e))
    print(f"host data base at : {app_settings.MILVUS_HOST}:{app_settings.MILVUS_PORT} not available!")

if connections.has_connection(alias=app_settings.MILVUS_ALIAS):
    print(f"Milvus connection to {app_settings.MILVUS_HOST}:{app_settings.MILVUS_PORT} on database {app_settings.MILVUS_DB} is established.")

Milvus connection to 127.0.0.1:20530 on database default is established.


In [11]:
connections.connect(
    alias=app_settings.MILVUS_ALIAS,
    host=app_settings.MILVUS_HOST,
    port=app_settings.MILVUS_PORT,
    db_name=app_settings.MILVUS_DB,
    secure=True,
    server_pem_path="/home/victor/gpt1/milvus/tls/server.pem",
    server_name="localhost"  # Must match SAN
)

In [16]:
# Создание полей для коллекции po_ea_v2 в векторной бд

from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, Function, FunctionType

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="vendor_text", dtype=DataType.VARCHAR, max_length=1000),
    FieldSchema(name="vendor_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="product_text", dtype=DataType.VARCHAR, max_length=1000),
    FieldSchema(name="product_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="row_number", dtype=DataType.INT32),
    FieldSchema(name="product_name", dtype=DataType.VARCHAR, max_length=1000),
    FieldSchema(name="vendor_name", dtype=DataType.VARCHAR, max_length=1000),
]

schema = CollectionSchema(fields=fields)

chunks_collection = Collection(name="po_ea_v2", schema=schema, using=app_settings.MILVUS_ALIAS)

In [17]:
%%time

# Загрузка данных в milvus

import numpy as np

for _, row in table.iterrows():
    print(_)
    vendor_str = '' if pd.isnull(row['Vendor']) else str(row['Vendor'])
    vendor_str = normalize_vendor_name(row['Vendor']).lower().strip()
    vendor_vector = emb_model.encode(vendor_str)['dense_vecs']
    product_str = normalize_product_name(row['Short Name'], vendor_str)
    product_vector = emb_model.encode(product_str)['dense_vecs']
    str_to_load = vendor_str + ' ' + product_str
    print(str_to_load)
    chunks_collection.insert([[vendor_str], [vendor_vector], [product_str], [product_vector], [row.name], [row['Short Name']], ['' if pd.isnull(row['Vendor']) else str(row['Vendor'])]])


0


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


abacom splan
1
becker mining systems ag becker
2
fast reports fastreport viewer
3
bresler winbres
4
microsemi corporation maxview storage manager
5
timhillone software h264webcam
6
jetbrains intellij idea
7
vostok nms fraktal
8
sovremennye programmnye tehnologii fireguard
9
belden hirschmann mice
10
d link des
11
tranzas navigator elektronno kartograficheskaja navigatsionno informatsionnaja sistema
12
indorsoft indorcad toro
13
bentley systems openroads designer
14
open source hex calc
15
google android studio
16
triada avia triada
17
custom is0001778
18
custom is0000057
19
custom is0000160
20
custom is0000732
21
custom is0000060
22
custom is0000440
23
kvantom kvant gibrid
24
bresler omp
25
skaut platforma
26
custom is0000892
27
custom is0000648
28
fabrikant elektronnaja torgovaja ploschadka
29
micromine pitram
30
mikrotikls dude
31
spaceteam holding spaceteam holding
32
cisco asa
33
cerner corporation cerner corporation
34
assotsiatsija vast smd4 oda
35
microsoft purview
36
cgtech sol

custom is0003371
299
red soft bd
300
open source avalonia
301
custom is0000254
302
givts gmk noril'skij nikel' zf custom is0003037
303
givts gmk noril'skij nikel' zf custom is0001758
304
custom is0000071
305
custom is0000445
306
abb ventyx minescape
307
sap portal
308
custom is0000188
309
evogen lis
310
infowatch endpoint security
311
palo alto networks pa 5430
312
parsec parsec
313
check point software technologies qls450
314
microsoft sql server
315
sitibank klient bank
316
adept adept
317
sonicwall nssp 13700
318
predprijatie elteks esr 1511 fstec
319
oracle health sciences
320
open source minio
321
solar sek'juriti inrights
322
siemens healthcare siemens healthcare
323
dentsply sirona sidexis 4 viewer
324
disc soft daemon tools lite
325
rockwell automation rsview factorytalk view machine edition
326
rocscience settle3
327
sd 3c card formatter
328
solarwinds worldwide dameware remote support
329
open source postgresql
330
paragon software image patrition creator hard disk manager 15

custom is0000409
575
givts gmk noril'skij nikel' zf custom is0001819
576
mettler toledo pk
577
custom is0000087
578
custom is0002201
579
uipath orchestrator
580
frendvork friendwork
581
vmware esxi server
582
custom is0000415
583
custom is0000977
584
custom is0000173
585
custom is0000371
586
custom is0000575
587
custom is0001969
588
ibm security identity governance intelligence
589
rtu mirea givts programmnyj kompleks 1 pk kvartal'naja
590
sisoft development stroitel'nye reshenija
591
rakuten viber
592
collins aerospace arinc
593
kodos globos
594
tehnopark dubna gensym g2
595
f5 networks big iq virtual edition centralized manager
596
sia zabbix server
597
kod bezopasnosti apksh kontinent
598
modx cms
599
vist grupp vg underground
600
ntts protej ats imswitch5
601
eshelon rubikon
602
vk tsifrovye tehnologii grc
603
eshelon pak rubikon
604
fortinet forticlient
605
predprijatie elteks esr 200 fstec
606
check point software technologies quantum rugged 1570r
607
esfi pak sistema sbora telem

...

ibm qradar siem
1677
custom is0000279
1678
tss diamond vpn fw 5211
1679
logika spt
1680
boss kadrovye sistemy hcm
1681
numa tehnologii edge
1682
dynamic marine systems b v sistema monitoringa elektrostatsii dms
1683
codec guide k lite pack
1684
schneider electrics electric connexium
1685
bimcollab zoom
1686
devart odacprofessional
1687
asus devicediscovery
1688
blackmagic design video assist
1689
imgburn imgburn
1690
rocscience rsdata
1691
javad gnss justin link
1692
gimp gimp
1693
power software poweriso
1694
salesforce tableau public
1695
howden gruppa ventsim design 5
1696
xerox corporation desktop print experience
1697
hangzhou hikvision digital technology vsplayer
1698
proxmox server solutions virtual environment
1699
finalwire aida64
1700
thermo scientific qtegra icap
1701
ooo sitiejr cityair plume
1702
open source prometheus
1703
ge digital webspace
1704
trukonf trueconf server
1705
rostelekom servis pechati
1706
ge digital iiot platform 1
1707
sap secure login client
1708
mega 

check point software technologies cloudguard
1948
roket kontrol suutp upravlenija proizvoditel'nost'ju mel'nits samoizmel'chenija ooo grkb
1949
rostelekom plagin esia ifcplugin
1950
fns sertifikat giis dmdk
1951
kredo dialog eshelon ii
1952
sisoft development otkrytye raspredelitel'nye ustrojstva
1953
lira sapr lira sapr
1954
spetstek trim
1955
kofax kofax
1956
deswik license server
1957
sap en
1958
sap assurance compliance
1959
tekla structures
1960
dassault systemes draftsight
1961
one identity manager
1962
palo alto networks pa 460
1963
gosstrojsmeta gosstrojsmeta
1964
landmark geographix discovery
1965
eshelon rubikon
1966
sonicwall tz300w
1967
everstage everstage
1968
fortinet fortigate vm16v
1969
autodesk autocad plant 3d
1970
eset protect platform
1971
kod bezopasnosti kontinent ipc
1972
anaplan scm
1973
informkontakt alfa hr
1974
visions healthcare visions healthcare
1975
custom pangolin
1976
skb psis scada eisa
1977
open source grafana
1978
epiroc underground manager
1979
dvdf

askon sistemy proektirovanija kompas
2216
salto systems salto
2217
dojche bank klient
2218
ajtransfers itransfers
2219
top kejs rasshirenie esep crypto extension brauzera
2220
custom is0003569
2221
custom is0003337
2222
custom is0000600
2223
dassault systemes quintiq
2224
barracuda message archiver 1050vx
2225
wallix admin bastion
2226
kiberprotekt kiber bekap
2227
wonderware information server
2228
sap bo
2229
sber sbermedai
2230
trastvers kub
2231
ibm webspher
2232
fortinet fortigate 60f
2233
fortinet fortixdr
2234
sisoft development tehnologicheskie shemy
2235
fortinet fortigate 100f
2236
trend micro vision one endpoint security
2237
kripto pro kriptopro office signature
2238
programbank abs
2239
spm soft spm soft
2240
astor rs scm
2241
oracle identity management
2242
numa tehnologii edge 25r
2243
moxa ics
2244
anylogic private cloud
2245
vmware workstation player
2246
modum lab mep
2247
tvins tehnologii zyfra eam
2248
too hr bot messenger
2249
disc soft daemon tools ultra
2250
daum

softbiznes reshenija logismart
2491
palo alto networks pa 3430
2492
sonicwall tz300
2493
iks24 x24 erp
2494
check point software technologies 1570
2495
bit crm
2496
micro focus arcsight
2497
autodesk 3ds max
2498
usachev andrej anatol'evich borej 3d
2499
palo alto networks pa vm
2500
smetawizard smetawizard
2501
sap cpq
2502
siemens simatic it
2503
rockitsoft ksed
2504
severstal' infokom nadezhnost'
2505
kripto pro kriptopro extension cades browser plug
2506
parallels international desktop
2507
tss diamond vpn fw 4101
2508
kod bezopasnosti kontinent ipc
2509
leksema lexema erp
2510
open source samba dc
2511
 custom is0004077
2512
moxa eds
2513
anaconda anaconda
2514
aveva wonderware information server portal
2515
inzhenernyj tsentr asi sistema avtomaticheskogo vzveshivanija
2516
watchkeeper watchkeeper
2517
codesys codesys
2518
ibs ekspertiza planeta bjudzhetirovanie
2519
icp das icp das
2520
corel corporation coreldraw graphics suite
2521
bartec sicherheits schaltanlagen iks500
2522
s

mongodb mongodb
2761
microsoft sharepoint designer
2762
microsoft windows server update services
2763
ibm lotus sametime connect
2764
adobe digital editions
2765
autodesk civil 3d
2766
intellektual'nye sistemy bezopasnosti securos
2767
custom is0001533
2768
custom asto kgmk
2769
custom is0000123
2770
custom is0000930
2771
custom is0000056
2772
custom is0000239
2773
custom is0001534
2774
custom is0002688
2775
custom is0000064
2776
custom is0003007
2777
saumi tsentr saumi
2778
hewlett packard hp service manager
2779
stek it stekenergo
2780
symantec helpdesk
2781
microsoft windows server ad federation services
2782
eos delo
2783
honeywell production manager
2784
infostroj kompleks a0
2785
custom is0000011
2786
custom is0003417
2787
custom is0000069
2788
custom is0000530
2789
ironport cisco esa
2790
1s billing
2791
1s upravlenie proektnoj organizatsiej
2792
dassault systemes surpac
2793
starlims lims
2794
serchinform siem
2795
fiix cmms
2796
akstim plan
2797
trend micro threat management s

In [18]:
# Индексирование полей

index_params = {
    "metric_type": "IP",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 1024},
}

chunks_collection.create_index("vendor_vector", index_params)
chunks_collection.create_index("product_vector", index_params)

Status(code=0, message=)

In [19]:
# Загрузка коллекции

chunks_collection.load()

# Составление мэппинга

In [20]:
po_table = pd.read_excel('ПО_04.09.25.xlsx')
po_table

,Полное наименование ПО,Версия,Вендор,Тип лицензирования,Семейство ПО,Категория ПО,Подкатегория ПО,Кол-во установок
0,Policy Platform,Unspecified,Microsoft,Компонент,Policy Platform,Утилиты,NaN,21291
1,Агент администрирования Kaspersky Security Center,Unspecified,Kaspersky Lab,Компонент,Агент администрирования Kaspersky Security Center,Информационная безопасность,NaN,21097
2,Visual C++ Redistributable 2013,2013,Microsoft,Бесплатное,Visual C++ Redistributable,Средства разработки,NaN,20909
3,Local Administrator Password Solution,Unspecified,Microsoft,Бесплатное,Local Administrator Password Solution,Сетевое администрирование,NaN,20713
4,Endpoint Configuration Manager Client,Unspecified,Microsoft,Коммерческое,Endpoint Configuration Manager Client,Сетевое администрирование,NaN,20097
...,...,...,...,...,...,...,...,...
10633,Photoshop CC (2015) - UNAUTHORIZED,CC (2015),Adobe,Коммерческое,Photoshop,Работа с контентом,Графические просмотрщики,1
10634,ShareX 13.7,13.7,ShareX,Бесплатное,ShareX,Работа с контентом,NaN,1
10635,Cognos 10,10,IBM,Коммерческое,Cognos,Отраслевое,Наука и анализ,1
10636,XnView 2.12,2.12,XnSoft,Коммерческое,XnView,Работа с контентом,Графические просмотрщики,1


In [24]:
# Функция для нахождения лучших результатов
# Сначала отсеиваем top_k_vendors по vendor_threshold
# Затем ищем в этом поднмножестве лучшее соответствие

from pymilvus import AnnSearchRequest

search_params = {"metric_type": "IP", "params": {"nprobe": 64}}
nprobe = 128

def get_embs_and_texts_with_vendor_filter(
    collection,
    vendor_query_vector: list, 
    product_query_vector: list, 
    vendor_threshold: float = 0.7,
    top_k_vendors: int = 1000,
    top_k_final: int = 100
):
    # Шаг 1: Найти ближайшие вендоры и сохранить их ID + distance
    vendor_search_params = {
        "data": [vendor_query_vector],
        "anns_field": "vendor_vector",
        "param": search_params,
        "limit": top_k_vendors,
        "output_fields": ["id"]  
    }
    
    vendor_results = collection.search(**vendor_search_params)
    
    # Создаём словарь: id -> vendor_similarity (distance)
    vendor_id_to_distance = {}
    for hit in vendor_results[0]:
        if hit.distance >= vendor_threshold:
            vendor_id_to_distance[hit.id] = hit.distance
    
    if not vendor_id_to_distance:
        return []

    # Шаг 2: Фильтруем поиск по этим ID
    allowed_ids = list(vendor_id_to_distance.keys())
    id_list_str = ",".join(str(i) for i in allowed_ids)
    expr = f"id in [{id_list_str}]"

    search_param_1 = {
        "data": [product_query_vector],
        "anns_field": "product_vector",
        "param": search_params,
        "limit": top_k_final,
        "expr": expr,
        "output_fields": ['id', 'row_number', 'product_name', 'vendor_name']
    }

    results = collection.search(**search_param_1)

    # Шаг 3: Собираем финальные результаты + добавляем vendor_distance
    final_results = []
    for hit in results[0]:
        record_id = hit.id
        final_results.append({
            'id': record_id,
            'product_distance': hit.distance,
            'vendor_distance': vendor_id_to_distance[record_id],
            'row_number': hit.entity.get("row_number"),
            'product_name': hit.entity.get("product_name"),
            'vendor_name': hit.entity.get("vendor_name"),
        })
    
    return final_results[:top_k_final]

In [26]:
%%time

# Составление окончательного мэппинга

table_map = pd.DataFrame(columns=['Вендор', 'Семейство ПО', 'vendor_name', 'product_name', 'vendor_distance', 'product_distance'])

for ind in po_table.index:
    vendor_str = normalize_vendor_name(po_table.loc[ind, 'Вендор']).lower().strip()
    vendor_vector = emb_model.encode(vendor_str)['dense_vecs']
    product_str = normalize_product_name(po_table.loc[ind, 'Семейство ПО'], vendor_str)
    product_vector = emb_model.encode(product_str)['dense_vecs']

    results = get_embs_and_texts_with_vendor_filter(chunks_collection, vendor_vector, product_vector)
    print('Запрос:', vendor_str, product_str)
    print()

    for result in results[:1]:
        print('Score:', result['product_distance'])
        print('Vendor Score:', result['vendor_distance'])
        print()
        print(result['product_name'])
        print(result['vendor_name'])
        print('@@@@@@@@@@@@@')
        print()
        
        dist = result['product_distance']
        
        str1 = '' if pd.isnull(po_table.loc[ind, 'Семейство ПО']) else str(po_table.loc[ind, 'Семейство ПО'])
        str2 = '' if pd.isnull(result['product_name']) else str(result['product_name'])
        vect_1 = emb_model.encode(str1.lower())['dense_vecs']
        vect_2 = emb_model.encode(str2.lower())['dense_vecs']
        
        if vect_1 @ vect_2.T > 0.9:
            dist = vect_1 @ vect_2.T
    
        table_map.loc[ind] = po_table.loc[ind, 'Вендор'], po_table.loc[ind, 'Семейство ПО'], result['vendor_name'], result['product_name'], result['vendor_distance'], dist
    if len(results) == 0:
        table_map.loc[ind] = po_table.loc[ind, 'Вендор'], po_table.loc[ind, 'Семейство ПО'], None, None, None, None


Запрос: microsoft policy platform

Score: 0.9999998807907104
Vendor Score: 0.9999998807907104

Microsoft Policy Platform
Microsoft
@@@@@@@@@@@@@

Запрос: kaspersky lab agent administrirovanija security center

Score: 0.6038332581520081
Vendor Score: 0.8838416337966919

Kaspersky Security Center
АО «Лаборатория Касперского»
@@@@@@@@@@@@@

Запрос: microsoft visual c redistributable

Score: 0.8627305030822754
Vendor Score: 0.9999998807907104

Microsoft Visual Studio 2015 Redistributable
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft local administrator password solution

Score: 1.0
Vendor Score: 0.9999998807907104

Microsoft Local Administrator Password Solution
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft endpoint configuration manager client

Score: 0.8998438119888306
Vendor Score: 0.9999998807907104

Microsoft Endpoint Configuration Manager
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft visual c redistributable

Score: 0.8627305030822754
Vendor Score: 0.9999998807907104

Microsoft Visual Studio 

Запрос: microsoft diskview

Score: 0.6660772562026978
Vendor Score: 0.9999998807907104

Microsoft Purview
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft logonsessions

Score: 0.5633288621902466
Vendor Score: 0.9999998807907104

Microsoft Remote Desktop Services
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft clockres

Score: 0.5623892545700073
Vendor Score: 0.9999998807907104

Microsoft Teams
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft rammap

Score: 0.5447981953620911
Vendor Score: 0.9999998807907104

Microsoft InfoPath
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft pendmoves

Score: 0.5181835293769836
Vendor Score: 0.9999998807907104

Microsoft PowerPoint
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft regdelnull

Score: 0.5031022429466248
Vendor Score: 0.9999998807907104

Microsoft Excel
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft registry usage ru

Score: 0.5547264814376831
Vendor Score: 0.9999998807907104

Microsoft Active Directory (AD)
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft sdelete

Score: 0.60

Запрос: mpc hc media player classic home cinema

Запрос: microsoft office infopath

Score: 0.885692834854126
Vendor Score: 0.9999998807907104

Microsoft InfoPath
Microsoft
@@@@@@@@@@@@@

Запрос: forcepoint one endpoint

Score: 0.8501560688018799
Vendor Score: 0.9999999403953552

FORCEPOINT ENDPOINT
Forcepoint
@@@@@@@@@@@@@

Запрос: revenera installshield

Запрос: microsoft winobj

Score: 0.5959048867225647
Vendor Score: 0.9999998807907104

Microsoft Windows
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft sysinternals process monitor

Score: 0.6669210195541382
Vendor Score: 0.9999998807907104

Microsoft Network Monitor
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft office components

Score: 0.7783621549606323
Vendor Score: 0.9999998807907104

Microsoft Office
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft office proofing tools

Score: 0.6767063140869141
Vendor Score: 0.9999998807907104

Microsoft Office
Microsoft
@@@@@@@@@@@@@

Запрос: oracle discoverer

Score: 0.5194054841995239
Vendor Score: 1.0



Запрос: nirsoft shellexview

Score: 0.6483089923858643
Vendor Score: 0.7127097845077515

XnView
XnSoft
@@@@@@@@@@@@@

Запрос: nir sofer usbdeview

Запрос: microsoft net runtime

Score: 0.6786800622940063
Vendor Score: 0.9999998807907104

Microsoft .NET Framework
Microsoft
@@@@@@@@@@@@@

Запрос: abbyy finereader

Score: 0.7533129453659058
Vendor Score: 1.0

ABBYY Finereader 10 Corporate Edition
ABBYY
@@@@@@@@@@@@@

Запрос: microsoft windows desktop runtime

Score: 0.697738528251648
Vendor Score: 0.9999998807907104

Microsoft Windows
Microsoft
@@@@@@@@@@@@@

Запрос: nanosoft nanocad spds

Score: 0.8145604133605957
Vendor Score: 1.0

nanoCAD
АО «Нанософт»
@@@@@@@@@@@@@

Запрос: nanosoft normacs client

Score: 0.8250558972358704
Vendor Score: 1.0

NormaCS
АО «Нанософт»
@@@@@@@@@@@@@

Запрос: microsoft net runtime pre releases

Score: 0.6367977857589722
Vendor Score: 0.9999998807907104

Microsoft .NET Core 2.1 Runtime & Hosting Bundle
Microsoft
@@@@@@@@@@@@@

Запрос: tektonit remote manipul

Запрос: sap output management system print application

Score: 0.5450936555862427
Vendor Score: 1.0

SAP SOLUTION MANAGER
SAP
@@@@@@@@@@@@@

Запрос: tektonit remote manipulator system

Запрос: autodesk dwg trueview

Score: 1.0
Vendor Score: 0.9999999403953552

DWG TrueView
Autodesk, Inc.
@@@@@@@@@@@@@

Запрос: ibm cognos

Score: 0.779589056968689
Vendor Score: 0.9999998807907104

Cognos Analytics
IBM
@@@@@@@@@@@@@

Запрос: codec guide k lite pack

Score: 0.9999998807907104
Vendor Score: 1.000000238418579

K-Lite Codec Pack
Codec Guide
@@@@@@@@@@@@@

Запрос: adobe acrobat

Score: 1.0
Vendor Score: 1.0

Adobe Acrobat
Adobe
@@@@@@@@@@@@@

Запрос: microsoft net framework

Score: 1.0
Vendor Score: 0.9999998807907104

Microsoft .NET Framework
Microsoft
@@@@@@@@@@@@@

Запрос: cisco anyconnect secure mobility client

Score: 0.7780482769012451
Vendor Score: 1.0

Cisco AnyConnect
Cisco
@@@@@@@@@@@@@

Запрос: microsoft endpoint configuration manager client

Score: 0.8998438119888306
Vendor Score:

...

Запрос: jam software treesize

Score: 0.44388818740844727
Vendor Score: 0.8024424910545349

Jama
Jama Software
@@@@@@@@@@@@@

Запрос: faststone soft capture

Score: 1.0
Vendor Score: 0.8855742812156677

FastStone Capture
FastStone
@@@@@@@@@@@@@

Запрос: jam software treesize

Score: 0.44388818740844727
Vendor Score: 0.8024424910545349

Jama
Jama Software
@@@@@@@@@@@@@

Запрос: krzysztof kowalczyk sumatrapdf

Запрос: adobe genuine software integrity service

Score: 0.4634236693382263
Vendor Score: 1.0

Adobe Digital Editions
Adobe
@@@@@@@@@@@@@

Запрос: schneider electric vijeo designer

Score: 1.0
Vendor Score: 0.979574978351593

Vijeo Designer
Schneider Electrics
@@@@@@@@@@@@@

Запрос: itva rusexcel

Score: 0.42671525478363037
Vendor Score: 0.7087321281433105

Landesk Management Suite
Ivanti
@@@@@@@@@@@@@

Запрос: adobe photoshop album

Score: 0.8480801582336426
Vendor Score: 1.0

Adobe Photoshop
Adobe
@@@@@@@@@@@@@

Запрос: teamviewer teamviewer

Запрос: hewlett packard enterprise sm

Запрос: mega international hopex

Score: 0.8885887861251831
Vendor Score: 0.8342174291610718

HOPEX Platform
MEGA
@@@@@@@@@@@@@

Запрос: mozilla firefox

Score: 0.8342707753181458
Vendor Score: 0.7833738327026367

Браузер Mozilla Firefox
Mozilla Foundation
@@@@@@@@@@@@@

Запрос: aol desktop

Score: 0.3646320104598999
Vendor Score: 0.7496715784072876

Драйвер ККТ
ООО «АТОЛ»
@@@@@@@@@@@@@

Запрос: seequent leapfrog geo

Score: 1.0
Vendor Score: 0.9999998807907104

LeapFrog GEO
Seequent
@@@@@@@@@@@@@

Запрос: polycom realpresence desktop

Score: 0.2694668173789978
Vendor Score: 0.7432112097740173

ZuluGIS
Политерм
@@@@@@@@@@@@@

Запрос: vmware view connection server

Score: 0.5644586086273193
Vendor Score: 0.9999999403953552

ESXi Server
VMware
@@@@@@@@@@@@@

Запрос: igor pavlov filezilla portable

Запрос: microsoft ado net entity framework

Score: 0.7037454843521118
Vendor Score: 0.9999998807907104

Microsoft .NET Framework
Microsoft
@@@@@@@@@@@@@

Запрос: icecream apps screen recorder



Запрос: iobit protected folder

Score: 0.3755568265914917
Vendor Score: 0.711910605430603

Cyberbit EDR
Cyberbit
@@@@@@@@@@@@@

Запрос: schneider electric psx ethernet rio dtm

Score: 0.47092169523239136
Vendor Score: 0.979574978351593

Schneider Electric CONNEXIUM
Schneider Electrics
@@@@@@@@@@@@@

Запрос: power software poweriso

Score: 1.0
Vendor Score: 1.0

PowerISO
Power Software
@@@@@@@@@@@@@

Запрос: stm otchet

Score: 0.4243520200252533
Vendor Score: 0.7285830974578857

ПК ООИ «ТЕЛЕСКАД»
ЗАО «ССТ»
@@@@@@@@@@@@@

Запрос: gnivts nalogoplatel'schik jul

Score: 1.0
Vendor Score: 1.0

Налогоплательщик ЮЛ
ГНИВЦ
@@@@@@@@@@@@@

Запрос: mobatek mobaxterm

Запрос: hexagon cloudworx autocad

Score: 0.4714670777320862
Vendor Score: 1.0000001192092896

Smart 3D
Hexagon
@@@@@@@@@@@@@

Запрос: national instruments ni labwindows cvi tdms library

Score: 0.35210275650024414
Vendor Score: 0.9999998807907104

National Instruments
National Instruments
@@@@@@@@@@@@@

Запрос: moscow exchange trade s

Запрос: broadcom installation manager

Score: 0.529672384262085
Vendor Score: 1.0

Symantec Endpoint Management
Broadcom Inc.
@@@@@@@@@@@@@

Запрос: maxon computer cinema 4d

Score: 0.9999998807907104
Vendor Score: 1.0000001192092896

Cinema 4D
MAXON COMPUTER GMBH
@@@@@@@@@@@@@

Запрос: microsoft identity manager synchronization service

Score: 0.6211835145950317
Vendor Score: 0.9999998807907104

Microsoft Windows Server Update Services
Microsoft
@@@@@@@@@@@@@

Запрос: microsoft office word viewer

Score: 0.8374148011207581
Vendor Score: 0.9999998807907104

Microsoft Office Word
Microsoft
@@@@@@@@@@@@@

Запрос: micro focus rubygem daemons

Score: 0.3714027404785156
Vendor Score: 1.0

Micro Focus ArcSight
Micro Focus
@@@@@@@@@@@@@

Запрос: microsoft system center data protection manager

Score: 0.7194849252700806
Vendor Score: 0.9999998807907104

Microsoft System Center Configuration Manager (SCCM)
Microsoft
@@@@@@@@@@@@@

Запрос: esri arcgis desktop

Score: 0.8035170435905457
Vendor Sc

Запрос: siemens simatic wincc runtime advanced

Score: 0.8531767725944519
Vendor Score: 1.0

Siemens Simatic WinCC
Siemens
@@@@@@@@@@@@@

Запрос: ispring solutions talkmaster

Запрос: corel winzip

Score: 0.4232860207557678
Vendor Score: 0.8640223741531372

CorelDRAW Graphics Suite
Corel Corporation
@@@@@@@@@@@@@

Запрос: hp display center

Запрос: kriptopro chromium gost

Score: 0.39943042397499084
Vendor Score: 0.8978161811828613

КриптоПро Extension for CAdES Browser Plug-in
ООО «КРИПТО-ПРО»
@@@@@@@@@@@@@

Запрос: trimble sketchup style builder

Score: 0.7848702669143677
Vendor Score: 1.0

SketchUp
Trimble Inc.
@@@@@@@@@@@@@

Запрос: freefilesync freefilesync

Запрос: contex nextimage

Запрос: opera software opera

Score: 0.4304336905479431
Vendor Score: 1.0

Браузер Opera
Opera Software
@@@@@@@@@@@@@

Запрос: chaos software v ray cinema 4d

Запрос: kodeks is tehekspert

Score: 0.855042040348053
Vendor Score: 1.0

ТехЭксперт
Кодекс
@@@@@@@@@@@@@

Запрос: adobe flash player

Score: 1

In [27]:
table_map

,Вендор,Семейство ПО,vendor_name,product_name,vendor_distance,product_distance
0,Microsoft,Policy Platform,Microsoft,Microsoft Policy Platform,1.0,1.0
1,Kaspersky Lab,Агент администрирования Kaspersky Security Center,АО «Лаборатория Касперского»,Kaspersky Security Center,0.883842,0.603833
2,Microsoft,Visual C++ Redistributable,Microsoft,Microsoft Visual Studio 2015 Redistributable,1.0,0.862731
3,Microsoft,Local Administrator Password Solution,Microsoft,Microsoft Local Administrator Password Solution,1.0,0.944162
4,Microsoft,Endpoint Configuration Manager Client,Microsoft,Microsoft Endpoint Configuration Manager,1.0,0.899844
...,...,...,...,...,...,...
10633,Adobe,Photoshop,Adobe,Adobe Photoshop,1.0,1.0
10634,ShareX,ShareX,ООО «САРЕКС»,Sarex Планирование,0.714869,0.663034
10635,IBM,Cognos,IBM,Cognos Analytics,1.0,0.779589
10636,XnSoft,XnView,XnSoft,XnView,1.0,1.0


# Выгрузка файла

In [28]:
table_map.to_excel('mapping v5.0.xlsx', index=False)